In [ ]:
%pdb

In [ ]:
from __future__ import print_function
from __future__ import division
import torch
import sys
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
from PIL import Image
from torch import autograd
from torch.autograd import Variable
import scipy.misc
from sklearn.metrics import confusion_matrix
from torch.utils.data import Dataset, DataLoader
from torchvision.utils import make_grid
import pickle as pkl
print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)
import tqdm
from tqdm import tqdm
import pickle

In [ ]:
data_file = "data/train_data/train_task_1_2.csv"
batch_size = 32
lr = 0.0001
num_epochs = 5
K = 100
total_q = 28000
total_s = 119000

In [ ]:
df = pd.read_csv(data_file)

In [ ]:
df.head()

In [ ]:
df.describe()

In [ ]:
df['QuestionId'].max()

In [ ]:
df['UserId'].max()

In [ ]:
class Question_Ans(Dataset):
    def __init__(self, filename, mode='train'):
        self.df = pd.read_csv(filename)
        self.questionid = self.df['QuestionId'].values
        self.userid = self.df['UserId'].values
        self.ans = self.df['IsCorrect'].values
        
        self.ans = 2*self.ans - 1
        
        self.length=len(self.ans)
        
        
        if(mode=='train'):
            start=int(0*self.length)
            end=int(0.8*self.length)
        else:
            start=int(0.8*self.length)
            end=int(1*self.length)
            
            
        self.questionid = self.questionid[start:end]
        self.userid = self.userid[start:end]
        self.ans = self.ans[start:end]
        
        self.length=len(self.ans)
        
    def __len__(self):
        return self.length
    
    def __getitem__(self, idx):
        
        qid = self.questionid[idx]
        uid = self.userid[idx]
        ans = self.ans[idx]
        return qid,uid,ans

In [ ]:
train_dataset = Question_Ans(filename=data_file,mode='train')
val_dataset = Question_Ans(filename=data_file,mode='val')

In [ ]:
train_dataloader = torch.utils.data.DataLoader(train_dataset,batch_size=batch_size, shuffle=True)
val_dataloader = torch.utils.data.DataLoader(val_dataset,batch_size=batch_size, shuffle=False)

In [ ]:
dataloader = {}
dataloader['train'] = train_dataloader
dataloader['val'] = val_dataloader

In [ ]:
dtype=torch.FloatTensor

In [ ]:
# Declare two matrices 
# Q = shape (total_q,K)
# U = shape (total_s,K)
Q = torch.nn.Embedding(total_q,K)
U = torch.nn.Embedding(total_s,K)

In [ ]:
# embedding = torch.nn.Embedding(5,8)

In [ ]:
# embedding(torch.LongTensor(np.array([3,4,3])))

In [ ]:
def get_qvector(questions):
    
#     q_list = []
#     for i in range(questions.shape[0]):
#         q_list.append(Q[i])
    
#     return torch.cat(q_list,dim=0)
    ans = Q(torch.LongTensor(questions))
    
    return ans

In [ ]:
def get_uvector(users):
    
#     u_list = []
#     for i in range(users.shape[0]):
#         u_list.append(U[i])
    
#     return torch.cat(u_list,dim=0)
#     print(users)
    ans = U(torch.LongTensor(users))
        
    return ans

In [ ]:
test1 = torch.randn(32,100)
test2 = torch.randn(32,100)
test1 = torch.unsqueeze(test1,1)
test2 = torch.unsqueeze(test2,2)
print(test1.shape)
print(test2.shape)
res = torch.bmm(test1,test2)
print(res.shape)
res = torch.squeeze(res)
print(res.shape)

In [ ]:
def get_score(qvectors,uvectors):
    
    q_unsq = torch.unsqueeze(qvectors, 1)
    u_unsq = torch.unsqueeze(uvectors, 2)
    score = torch.bmm(q_unsq,u_unsq)
    score = torch.squeeze(score)
    return score

In [ ]:
params_to_update = list(Q.parameters()) + list(U.parameters())

In [ ]:
criterion = nn.MSELoss()
optimizer = optim.SGD(params=params_to_update,lr=lr, momentum=0.9)

In [ ]:
best_loss = 1e9
for epoch in range(num_epochs):
    print('Epoch {}/{}'.format(epoch, num_epochs - 1))
    print('-' * 10)
    # Each epoch has a training and validation phase
    for phase in ['train', 'val']:
        
            
        running_loss = 0.0
        
        # Iterate over data.
        cnt = 0
#         print("*********** entered ",phase,"*********************")
        for questions,users,ans in tqdm(dataloader[phase]):
            
#             print("************ loaded data of one batch*******")
            
            cnt = cnt + 1
            ans = torch.tensor(ans)
            ans = ans.type(dtype)
            
            qvectors = get_qvector(questions)
            uvectors = get_uvector(users)
            
            
            # zero the parameter gradients
            optimizer.zero_grad()
            # forward
            
            scores = get_score(qvectors,uvectors)
            
            loss = criterion(scores,ans)
            
            if phase == 'train':
                loss.backward()
                optimizer.step()
        
            # statistics
            running_loss += loss.item()
            
            # compute correct
            # running_corrects += torch.sum(preds == labels.data)
            
            if (cnt % 1000 == 0 ) :
                print('Batch {} {} Loss: {:.4f} '.format(str(cnt),phase, running_loss/(cnt*batch_size)))
            

        epoch_loss = running_loss / len(dataloader[phase].dataset)

        print(' {} Loss: {:.4f} '.format(phase, epoch_loss))
        
        # deep copy the model
        if phase == 'val' and epoch_loss < best_loss:
            best_loss = epoch_loss
            # best_model_wts = copy.deepcopy(model.state_dict())
            # do something about saving best matrices
    q_file = 'models/simple_fact_q' + str(epoch+1) + '_.pkl'
    u_file = 'models/simple_fact_u' + str(epoch+1) + '_.pkl'
    q_f = open(q_file, 'wb')
    u_f = open(u_file, 'wb')
    pickle.dump(Q,q_f)
    pickle.dump(U,u_f)
    print()
time_elapsed = time.time() - since
print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
print('Best val Loss: {:4f}'.format(best_loss))

In [ ]:
q_file = 'models/simple_fact_q.pkl'
u_file = 'models/simple_fact_u.pkl'

In [ ]:
q_f = open(q_file, 'wb')
u_f = open(u_file, 'wb')

In [ ]:
import pickle

In [ ]:
pickle.dump(Q,q_f)
pickle.dump(U,u_f)